In [ ]:
from operator import itemgetter

def print_table(processes):
    print("\nProcess | Arrival | Burst | Priority | Completion | Turnaround | Waiting")
    for p in processes:
        print(f"{p['pid']:>7} | {p['arrival']:>7} | {p['burst']:>5} | {p.get('priority','-'):>8} |"
              f" {p['completion']:>10} | {p['turnaround']:>10} | {p['waiting']:>7}")

def fcfs(processes):
    processes.sort(key=lambda x: x['arrival'])
    current_time = 0
    for p in processes:
        current_time = max(current_time, p['arrival'])
        p['start'] = current_time
        p['completion'] = current_time + p['burst']
        current_time += p['burst']
        p['turnaround'] = p['completion'] - p['arrival']
        p['waiting'] = p['turnaround'] - p['burst']
    print("\n--- FCFS Scheduling ---")
    print_table(processes)

def sjf(processes):
    processes.sort(key=lambda x: (x['arrival'], x['burst']))
    completed = []
    current_time = 0
    while processes:
        available = [p for p in processes if p['arrival'] <= current_time]
        if not available:
            current_time = processes[0]['arrival']
            continue
        shortest = min(available, key=lambda x: x['burst'])
        processes.remove(shortest)
        shortest['start'] = current_time
        shortest['completion'] = current_time + shortest['burst']
        current_time += shortest['burst']
        shortest['turnaround'] = shortest['completion'] - shortest['arrival']
        shortest['waiting'] = shortest['turnaround'] - shortest['burst']
        completed.append(shortest)
    print("\n--- SJF Scheduling ---")
    print_table(completed)

def priority_scheduling(processes):
    processes.sort(key=lambda x: (x['arrival'], x['priority']))
    completed = []
    current_time = 0
    while processes:
        available = [p for p in processes if p['arrival'] <= current_time]
        if not available:
            current_time = processes[0]['arrival']
            continue
        highest = min(available, key=lambda x: x['priority'])
        processes.remove(highest)
        highest['start'] = current_time
        highest['completion'] = current_time + highest['burst']
        current_time += highest['burst']
        highest['turnaround'] = highest['completion'] - highest['arrival']
        highest['waiting'] = highest['turnaround'] - highest['burst']
        completed.append(highest)
    print("\n--- Priority Scheduling ---")
    print_table(completed)

def round_robin(processes, time_quantum):
    from collections import deque
    queue = deque()
    current_time = 0
    remaining = {p['pid']: p['burst'] for p in processes}
    arrival_map = {p['pid']: p['arrival'] for p in processes}
    pid_map = {p['pid']: p for p in processes}
    completed = []
    arrived = []
    processes.sort(key=lambda x: x['arrival'])
    i = 0
    while i < len(processes) and processes[i]['arrival'] <= current_time:
        queue.append(processes[i]['pid'])
        arrived.append(processes[i]['pid'])
        i += 1

    while queue or i < len(processes):
        if not queue:
            current_time = processes[i]['arrival']
            while i < len(processes) and processes[i]['arrival'] <= current_time:
                if processes[i]['pid'] not in arrived:
                    queue.append(processes[i]['pid'])
                    arrived.append(processes[i]['pid'])
                i += 1
        pid = queue.popleft()
        exec_time = min(time_quantum, remaining[pid])
        remaining[pid] -= exec_time
        current_time += exec_time

        while i < len(processes) and processes[i]['arrival'] <= current_time:
            if processes[i]['pid'] not in arrived:
                queue.append(processes[i]['pid'])
                arrived.append(processes[i]['pid'])
            i += 1

        if remaining[pid] == 0:
            p = pid_map[pid]
            p['completion'] = current_time
            p['turnaround'] = p['completion'] - p['arrival']
            p['waiting'] = p['turnaround'] - p['burst']
            completed.append(p)
        else:
            queue.append(pid)

    completed.sort(key=lambda x: x['pid'])
    print("\n--- Round Robin Scheduling ---")
    print_table(completed)

def get_process_input():
    n = int(input("Enter number of processes: "))
    processes = []
    for i in range(n):
        p = {
            'pid': f'P{i+1}',
            'arrival': int(input(f"Arrival time of P{i+1}: ")),
            'burst': int(input(f"Burst time of P{i+1}: "))
        }
        processes.append(p)
    return processes

def get_priority_input():
    processes = get_process_input()
    for p in processes:
        p['priority'] = int(input(f"Priority of {p['pid']} (Lower = Higher priority): "))
    return processes

def main():
    while True:
        print("\nCPU Scheduling Algorithms:")
        print("1. First Come First Serve (FCFS)")
        print("2. Shortest Job First (SJF)")
        print("3. Priority Scheduling")
        print("4. Round Robin (RR)")
        print("0. Exit")
        choice = input("Choose algorithm (0-4): ")
        
        if choice == '1':
            fcfs(get_process_input())
        elif choice == '2':
            sjf(get_process_input())
        elif choice == '3':
            priority_scheduling(get_priority_input())
        elif choice == '4':
            tq = int(input("Enter time quantum: "))
            round_robin(get_process_input(), tq)
        elif choice == '0':
            print("Exiting...")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()



CPU Scheduling Algorithms:
1. First Come First Serve (FCFS)
2. Shortest Job First (SJF)
3. Priority Scheduling
4. Round Robin (RR)
0. Exit


Choose algorithm (0-4):  1
Enter number of processes:  5
Arrival time of P1:  1
Burst time of P1:  5
Arrival time of P2:  2
Burst time of P2:  7
Arrival time of P3:  3
Burst time of P3:  5
Arrival time of P4:  9
Burst time of P4:  10
Arrival time of P5:  3
Burst time of P5:  8



--- FCFS Scheduling ---

Process | Arrival | Burst | Priority | Completion | Turnaround | Waiting
     P1 |       1 |     5 |        - |          6 |          5 |       0
     P2 |       2 |     7 |        - |         13 |         11 |       4
     P3 |       3 |     5 |        - |         18 |         15 |      10
     P5 |       3 |     8 |        - |         26 |         23 |      15
     P4 |       9 |    10 |        - |         36 |         27 |      17

CPU Scheduling Algorithms:
1. First Come First Serve (FCFS)
2. Shortest Job First (SJF)
3. Priority Scheduling
4. Round Robin (RR)
0. Exit


Choose algorithm (0-4):  1
Enter number of processes:  4
Arrival time of P1:  1
Burst time of P1:  8
Arrival time of P2:  5
Burst time of P2:  10
Arrival time of P3:  3
Burst time of P3:  9
Arrival time of P4:  2
Burst time of P4:  14



--- FCFS Scheduling ---

Process | Arrival | Burst | Priority | Completion | Turnaround | Waiting
     P1 |       1 |     8 |        - |          9 |          8 |       0
     P4 |       2 |    14 |        - |         23 |         21 |       7
     P3 |       3 |     9 |        - |         32 |         29 |      20
     P2 |       5 |    10 |        - |         42 |         37 |      27

CPU Scheduling Algorithms:
1. First Come First Serve (FCFS)
2. Shortest Job First (SJF)
3. Priority Scheduling
4. Round Robin (RR)
0. Exit
